In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [16]:
df = pd.read_csv("/kaggle/input/day5-sample/Day5_sample.txt",header=None, sep=':')
df

,0,1
0,seeds,79 14 55 13
1,seed-to-soil map,NaN
2,50 98 2,NaN
3,52 50 48,NaN
4,soil-to-fertilizer map,NaN
5,0 15 37,NaN
6,37 52 2,NaN
7,39 0 15,NaN
8,fertilizer-to-water map,NaN
9,49 53 8,NaN


In [17]:
seed_df = pd.DataFrame()
seed_df['seed_1'] = df.iloc[0,1].split()
seed_df['seed_2'] = df.iloc[0,1].split()
seed_df = seed_df.astype('int64')

%%time
# this way
dict_map = {}
for idx in tqdm(range(1, len(df))):
    #print('idx :',idx)
    if df.at[idx,0][0].isalpha():
        dict_map[df.at[idx,0]]={}
        i = 0
        idx2 = idx + 1
        #print('idx2 :',idx2)
        try:
            while df.at[idx2,0][0].isdigit():                
                    list_row = [int(x) for x in df.at[idx2,0].split()]
                    zip1 = list(range(list_row[1], list_row[1]+list_row[2]))
                    zip2 = list(range(list_row[0], list_row[0]+list_row[2]))
                    #print('dict_map avant: ',dict_map)
                    #print('zip : ',{df.at[idx,0]:dict(zip(zip1,zip2))})
                    # Fusionnez les deux dictionnaires
                    for key, value in {df.at[idx,0]:dict(zip(zip1,zip2))}.items():
                        dict_map[key] = {**dict_map[key],**value}
                    #print('dict_map après:',dict_map)
                    idx2 += 1
        except:
            break
        #print(dict_map)


In [ ]:
%%time
# that way
dict_map = {}
idx = 1

while idx < len(df) and df.at[idx, 0][0].isalpha():
    key = df.at[idx, 0]
    dict_map[key] = {}
    idx2 = idx + 1

    while idx2 < len(df) and df.at[idx2, 0][0].isdigit():
        list_row = [int(x) for x in df.at[idx2, 0].split()]
        '''
        zip1 = list(range(list_row[1], list_row[1] + list_row[2]))
        zip2 = list(range(list_row[0], list_row[0] + list_row[2]))
        dict_map[key].update(dict(zip(zip1, zip2)))
        idx2 += 1
        '''
        # Supposez que list_row est déjà défini
        # list_row = [start_value, end_value, step]

        start_value1, end_value1, step1 = list_row[1], list_row[1] + list_row[2], 1
        start_value2, end_value2, step2 = list_row[0], list_row[0] + list_row[2], 1

        # Utilisez la fonction range directement dans update
        dict_map[key].update({i: j for i, j in zip(range(start_value1, end_value1, step1), range(start_value2, end_value2, step2))})

    idx += 1

# Affichez le résultat final
print(dict_map)

# that other way
# Extract the relevant columns outside the loop
column_0 = df[0]
column_1 = df[1]

dict_map = {}
idx = 1

while idx < len(df) and column_0[idx][0].isalpha():
    key = column_0[idx]
    dict_map[key] = {}
    idx2 = idx + 1

    while idx2 < len(df) and column_0[idx2][0].isdigit():
        list_row = [int(x) for x in column_0[idx2].split()]

        start_value1, end_value1, step1 = list_row[1], list_row[1] + list_row[2], 1
        start_value2, end_value2, step2 = list_row[0], list_row[0] + list_row[2], 1

        dict_map[key].update({i: j for i, j in zip(range(start_value1, end_value1, step1), range(start_value2, end_value2, step2))})
        idx2 += 1

    idx += 1

# Affichez le résultat final
print(dict_map)

%%time
# this way
data_frame = {}
for k in tqdm(dict_map):
    data_frame[k] = pd.DataFrame(dict_map[k],index=[0]).T
    #print(data_frame[k])

In [ ]:
%%time
# that way
data_frame = {k: pd.DataFrame(v, index=[0]).T for k, v in dict_map.items()}

In [ ]:
for k in tqdm(dict_map):
    data_frame[k] = data_frame[k].reset_index(drop=False)
    col1 = f"{k}{'_1'}"
    col2 = f"{k}{'_2'}"
    data_frame[k][col1] = data_frame[k]['index']
    data_frame[k][col2] = data_frame[k][0]
    data_frame[k].drop(columns=['index',0], inplace=True)

In [ ]:
seed_df

In [ ]:
dict_map = {**{'seed_to_seed': seed_df}, **dict_map}

In [ ]:
dict_map.keys()

In [ ]:
data_frame = {**{'seed_to_seed': seed_df}, **data_frame}

In [ ]:
data_frame['seed-to-soil map'].head(2)

In [ ]:
df

In [ ]:
result = data_frame[list(dict_map.keys())[0]]
for i in tqdm(range(1,len(dict_map))):
    # Fusionnez le DataFrame sur sa colonne 'key' avec l'autre DataFrame sur son index
    df1 = result#data_frame[list(dict_map.keys())[i]]
    df2 = data_frame[list(dict_map.keys())[i]]
    #print('result before : ',result)
    #print('df1 : ',df1)
    result = pd.merge(df1, df2, how='left',left_on=str(result.columns[-1]), right_on=str(df2.columns[0]))
    print('result before : ',result.head(5))
    for i  in result.index:
          #print('result.iloc[i,-1]', result.iloc[i,-1])                      
        if result.iloc[i,-1] != result.iloc[i,-1]:
            #print('result.iloc[i,-1] NaN détecté')
            result.iloc[i,-1] = result.iloc[i,-3]
            #print('result.iloc[i,-1] changé en : ', result.iloc[i,-1])
            
    print('result after : ',result.head())
# Affichez le résultat
result = pd.DataFrame(result)
#result.columns

In [ ]:
min(result.iloc[:,-1])